# Iteration 4: warm-up period

## Plain English Summary

This is a non-terminating simulation.  The next step is to implement a warm-up period parameter for the model.  This is user settable.




## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [2]:
display(Markdown("inputs/iteration_4.md"))

modify the code as follows

the model should include a user settable warm up period. This defaults to 1 month.

model run length should be renamed results collection period and have the default value of 12 months.  

The total run length of the model is the warm up period plus the results collection period.

the Experiment class should also accept a parameter for the warm up period

Show all of the model code: this includes all patient generator functions in the CCModel class and all code in the Experiment class.

## Date and time of input to LLM

> 1840 29th November 2023 

## Observations on the LLM Output


* The warm-up period has been added to `Experiment` as a parameter.
* `run_length` has been correctly renamed to `results_collection_period`
* The model is run for the `results_collection_period` + `warm_up_period`
* At the moment the warm-up period is not used anywhere else in the model.

## LLM Output 1:

In this modified code, the Experiment class accepts a parameter for the warm up period, and the run method of the CCUModel class refers to self.env instead of the variable env. The model run length is renamed results collection period and has the default value of 12 months. The total run length of the model is the warm up period plus the results collection period

In [3]:
import simpy
import numpy as np

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)


In [4]:
# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

Patient 1 arrived from the Wards at time 12.3457005039859
Patient 1 admitted to critical care bed at time 12.3457005039859
Elective surgery patient 2 arrived at time 22.644902770733474
Elective surgery patient 2 admitted to critical care bed at time 22.644902770733474
Patient 3 arrived from Emergency surgery at time 25.573891890485392
Patient 3 admitted to critical care bed at time 25.573891890485392
Patient 4 arrived from Accident and Emergency at time 32.67170714874762
Patient 4 admitted to critical care bed at time 32.67170714874762
Elective surgery patient 5 arrived at time 36.89882994237517
Elective surgery patient 5 admitted to critical care bed at time 36.89882994237517
Patient 6 arrived from Accident and Emergency at time 50.08984128104649
Patient 6 admitted to critical care bed at time 50.08984128104649
Patient 7 arrived from other hospitals at time 51.79610196733608
Patient 7 admitted to critical care bed at time 51.79610196733608
Elective surgery patient 8 arrived at time 52

## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [5]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [6]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [7]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [8]:
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 13.793871106830004
Patient 1 admitted to critical care bed at time 13.793871106830004
Patient 1 discharged from critical care bed at time 19.264066154687182
Critical care bed is available for next patient at time 24.264066154687182
Patient 2 arrived from Accident and Emergency at time 49.91757271153867
Patient 2 admitted to critical care bed at time 49.91757271153867
Patient 3 arrived from Accident and Emergency at time 85.95030355769869
Patient 3 admitted to critical care bed at time 85.95030355769869
Patient 3 discharged from critical care bed at time 96.89223987280607
Critical care bed is available for next patient at time 101.89223987280607
Patient 4 arrived from Accident and Emergency at time 157.41972912799747
Patient 4 admitted to critical care bed at time 157.41972912799747
Patient 5 arrived from Accident and Emergency at time 165.28142246925495
Patient 5 admitted to critical care bed at time 165.28142246925495
Patient 5 dis

### Test 2: Extreme value test 2

PASS

In [9]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [10]:
extreme_value_test_2()

Elective surgery patient 1 arrived at time 17.841954920945458
Elective surgery patient 1 admitted to critical care bed at time 17.841954920945458
Elective surgery patient 2 arrived at time 36.30002490288469
Elective surgery patient 2 admitted to critical care bed at time 36.30002490288469
Elective surgery patient 3 arrived at time 54.69981683534241
Elective surgery patient 3 admitted to critical care bed at time 54.69981683534241
Elective surgery patient 3 discharged from critical care bed at time 64.50864350697536
Critical care bed is available for next patient at time 69.50864350697536
Elective surgery patient 4 arrived at time 74.92364530928282
Elective surgery patient 4 admitted to critical care bed at time 74.92364530928282
Elective surgery patient 5 arrived at time 92.04837772081746
Elective surgery patient 5 admitted to critical care bed at time 92.04837772081746
Elective surgery patient 6 arrived at time 108.86838220582104
Elective surgery patient 6 admitted to critical care be

### Test 3: Different run length

PASS

In [13]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [14]:
test_results_collection_period(10)

Patient 1 arrived from the Wards at time 12.96756683713932
Patient 1 admitted to critical care bed at time 12.96756683713932
Elective surgery patient 2 arrived at time 18.158322719972215
Elective surgery patient 2 admitted to critical care bed at time 18.158322719972215
Patient 3 arrived from the Wards at time 18.63917649413765
Patient 3 admitted to critical care bed at time 18.63917649413765
Patient 4 arrived from Accident and Emergency at time 19.589649080649295
Patient 4 admitted to critical care bed at time 19.589649080649295
Patient 5 arrived from the Wards at time 22.1268875909895
Patient 5 admitted to critical care bed at time 22.1268875909895
Patient 6 arrived from Emergency surgery at time 29.53470618985853
Patient 6 admitted to critical care bed at time 29.53470618985853
Patient 7 arrived from Emergency surgery at time 32.195365748307054
Patient 7 admitted to critical care bed at time 32.195365748307054
Elective surgery patient 8 arrived at time 37.12557346596353
Elective sur

730

### Test 4: Extreme value test 3

PASS.

In [16]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [17]:
extreme_value_test_3()

Patient 1 arrived from Emergency surgery at time 2.510009074608789
Patient 1 admitted to critical care bed at time 2.510009074608789
Patient 2 arrived from Accident and Emergency at time 9.113211670063446
Patient 3 arrived from Emergency surgery at time 11.527201522444608
Elective surgery patient 4 arrived at time 15.598945320419698
Elective surgery for patient 4 cancelled due to no available critical care beds at time 15.598945320419698
Patient 5 arrived from Accident and Emergency at time 16.75658717917721
Patient 6 arrived from Accident and Emergency at time 17.800350215421926
Patient 6 discharged from critical care bed at time 17.931927811135694
Critical care bed is available for next patient at time 22.931927811135694
Patient 6 admitted to critical care bed at time 22.931927811135694
Elective surgery patient 7 arrived at time 28.68783092403146
Elective surgery for patient 7 cancelled due to no available critical care beds at time 28.68783092403146
Patient 8 arrived from Accident a

### Test 5: Extreme value test 4

PASS

In [18]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [19]:
extreme_value_test_4()

Patient 1 arrived from other hospitals at time 5.227206083659815
Patient 1 admitted to critical care bed at time 5.227206083659815
Patient 2 arrived from Accident and Emergency at time 12.13575580066347
Patient 2 admitted to critical care bed at time 12.13575580066347
Elective surgery patient 3 arrived at time 21.166009039061446
Elective surgery patient 3 admitted to critical care bed at time 21.166009039061446
Patient 4 arrived from Accident and Emergency at time 27.637856816193732
Patient 4 admitted to critical care bed at time 27.637856816193732
Patient 5 arrived from other hospitals at time 27.64852438894139
Patient 5 admitted to critical care bed at time 27.64852438894139
Patient 6 arrived from the Wards at time 28.66353007794818
Patient 6 admitted to critical care bed at time 28.66353007794818
Elective surgery patient 7 arrived at time 35.65495351160668
Elective surgery patient 7 admitted to critical care bed at time 35.65495351160668
Patient 8 arrived from other hospitals at tim

### Test 6: Vary Warm-up parameter

PASS

In [24]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [23]:
test_warmup_period(0)

Patient 1 arrived from the Wards at time 16.411472810859685
Patient 1 admitted to critical care bed at time 16.411472810859685
Patient 2 arrived from the Wards at time 16.466553491700807
Patient 2 admitted to critical care bed at time 16.466553491700807
Elective surgery patient 3 arrived at time 16.90442544182514
Elective surgery patient 3 admitted to critical care bed at time 16.90442544182514
Patient 4 arrived from Accident and Emergency at time 17.362071530177303
Patient 4 admitted to critical care bed at time 17.362071530177303
Patient 5 arrived from other hospitals at time 21.51339138765345
Patient 5 admitted to critical care bed at time 21.51339138765345
Elective surgery patient 5 discharged from critical care bed at time 21.75851715475729
Patient 5 discharged from critical care bed at time 25.79741346488779
Critical care bed is available for next patient at time 26.75851715475729
Critical care bed is available for next patient at time 30.79741346488779
Elective surgery patient 6

(8640, 8640)